In [40]:
import json
import time
import numpy as np
import trimesh

# Global constants and mappings (as before)
SBE_COLORS = {
    # Walls: a warm off-white with subtle cream tones.
    "walls": [0.8, 0.8, 0.8],
    # Interior walls: almost as light as exterior, with a slightly softer tone.
    "interior_walls": [0.99, 0.98, 0.97],
    # Roofs: even lighter than walls for a very gentle contrast.
    "roofs": [0.2, 0.2, 0.2],
    # Ceilings: crisp pure white.
    "ceilings": [1.0, 1.0, 1.0],
    # Exterior floors: a light, cool concrete tone.
    "exterior_floors": [0.90, 0.88, 0.85],
    # Interior floors: a warm, soft wood-inspired tone.
    "interior_floors": [0.95, 0.92, 0.90],
    # Air walls: nearly pure white to emphasize the envelope.
    "air_walls": [0.99, 0.99, 0.99],
    # Apertures (windows): a pastel blueish-grey for a modern, airy feel.
    "apertures": [0.7, 0.75, 0.92],
    # Interior apertures (skylights, etc.): a slightly lighter version.
    "interior_apertures": [0.85, 0.88, 0.92],
    # Doors: kept dark as accent details.
    "doors": [0.20, 0.18, 0.16],
    # Interior doors: a soft dark gray.
    "interior_doors": [0.30, 0.28, 0.26],
    # Outdoor shades: a light neutral gray.
    "outdoor_shades": [0.65, 0.65, 0.65],
    # Indoor shades: a very light, airy gray.
    "indoor_shades": [0.75, 0.75, 0.75],
    # Shade meshes: a balanced mid-gray, light and unobtrusive.
    "shade_meshes": [0.55, 0.55, 0.55],
}
FACE_TYPE_MAPPING = {
    "Wall": "walls",
    "RoofCeiling": "roofs",
    "Floor": "interior_floors",  # Default; updated later based on boundary conditions
}
APERTURE_OFFSET = 0.1

def get_rotation_matrix(rotate = False):
    """Return a rotation matrix (rotate -90° about X-axis)."""
    if rotate:
        return trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])
    else:
        return np.eye(4)

def compute_faces(num_vertices):
    """Return triangle fan faces for a polygon with num_vertices."""
    return np.array([[0, i, i + 1] for i in range(1, num_vertices - 1)], dtype=np.int64)

# --- New vectorized processing functions ---
def process_aperture_vectorized(aperture, face_type, rotation_matrix):
    """
    Process an aperture in a vectorized way:
      - Rotate its vertices
      - Compute its normal (from the first triangle)
      - Offset all vertices along that normal
      - Compute local faces for the aperture polygon
    Returns (vertices, faces, color, category) or (None, None, None, None) if invalid.
    """
    aperture_geometry = aperture.get("geometry", {})
    boundary = aperture_geometry.get("boundary", [])
    if len(boundary) < 3:
        return None, None, None, None

    vertices = np.array(boundary, dtype=np.float32)
    # Rotate all vertices at once.
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))
    # Compute the normal from the first triangle (vectorized)
    tri = rotated[0:3].reshape(1, 3, 3)
    normal = trimesh.triangles.normals(tri)[0]
    normal = trimesh.unitize(normal)
    # Offset all vertices along the normal.
    rotated += APERTURE_OFFSET * normal

    # Determine category and color
    if face_type == "Wall":
        category = "apertures"
    elif face_type == "RoofCeiling":
        category = "interior_apertures"
    else:
        category = "apertures"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    return rotated, local_faces, color, category

def process_face_vectorized(face, rotation_matrix):
    """
    Process a face by:
      - Rotating its vertices in one batch,
      - Computing local faces,
      - Determining its category,
      - And processing its apertures in the same vectorized fashion.
    Returns a dictionary mapping category names to lists of (vertices, faces, color) tuples.
    """
    results = {cat: [] for cat in SBE_COLORS.keys()}
    geometry = face.get("geometry", {})
    boundary = geometry.get("boundary", [])
    if len(boundary) < 3:
        return results

    vertices = np.array(boundary, dtype=np.float32)
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))

    face_type = face.get("face_type", "Wall")
    category = FACE_TYPE_MAPPING.get(face_type, "walls")
    bc = face.get("boundary_condition", {}).get("type", "")
    if face_type == "Floor":
        category = "exterior_floors" if bc == "Ground" else "interior_floors"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    results[category].append((rotated, local_faces, color))

    # Process apertures attached to the face.
    for aperture in face.get("apertures", []):
        ap_vertices, ap_faces, ap_color, ap_cat = process_aperture_vectorized(aperture, face_type, rotation_matrix)
        if ap_vertices is not None:
            results[ap_cat].append((ap_vertices, ap_faces, ap_color))
    return results

def process_room_vectorized(room, rotation_matrix):
    """
    Process all faces in a room, returning a dictionary mapping category names
    to lists of (vertices, faces, color) tuples.
    """
    room_results = {cat: [] for cat in SBE_COLORS.keys()}
    for face in room.get("faces", []):
        face_dict = process_face_vectorized(face, rotation_matrix)
        for cat, items in face_dict.items():
            room_results[cat].extend(items)
    return room_results

def combine_category_geometry(items):
    """
    Given a list of items (each a tuple of (vertices, faces, color)) for one category,
    combine them into one large mesh using vectorized concatenation.
    Assumes all items in the list share the same vertex color.
    Returns a trimesh.Trimesh object.
    """
    if not items:
        return None

    all_vertices = []
    all_faces = []
    offset = 0
    for vertices, faces, _ in items:
        all_vertices.append(vertices)
        all_faces.append(faces + offset)
        offset += len(vertices)
    vertices_combined = np.vstack(all_vertices)
    faces_combined = np.vstack(all_faces)
    mesh = trimesh.Trimesh(vertices=vertices_combined, faces=faces_combined, process=False)
    # Use the color from the first item (they are assumed identical for a given category).
    mesh.visual.vertex_colors = items[0][2]
    return mesh


# --- Configuration & Data Loading ---
path_to_json = '../public/urban_district.hbjson'
output_path = path_to_json.replace(".hbjson", ".glb")
rotation_matrix = get_rotation_matrix()

with open(path_to_json, 'r') as f:
    hbjson = json.load(f)
rooms = hbjson.get("rooms", [])

# Instead of creating many small meshes, we accumulate geometry per category.
combined_results = {cat: [] for cat in SBE_COLORS.keys()}

t0 = time.perf_counter()
for room in rooms:
    room_geom = process_room_vectorized(room, rotation_matrix)
    for cat, items in room_geom.items():
        combined_results[cat].extend(items)
t1 = time.perf_counter()
print(f"Accumulated geometry from all rooms in {t1 - t0:.4f} seconds")

# Build one big mesh per category by concatenating vertices and faces.
meshes = []
for cat, items in combined_results.items():
    mesh = combine_category_geometry(items)
    if mesh is not None:
        meshes.append(mesh)

# Finally, combine all category meshes into one mesh.
if meshes:
    final_mesh = trimesh.util.concatenate(meshes)
    t2 = time.perf_counter()
    final_mesh.export(output_path)
    t3 = time.perf_counter()
    print(f"Exported GLB file: {output_path}")
    print(f"Mesh concatenation took {t2 - t1:.4f} seconds, export took {t3 - t2:.4f} seconds")
else:
    print("No valid geometry found in HBJSON.")




Accumulated geometry from all rooms in 1.4580 seconds
Exported GLB file: ../public/urban_district.glb
Mesh concatenation took 0.0999 seconds, export took 0.0222 seconds


In [1]:
import json
import time
import numpy as np
import trimesh
from trimesh.visual.material import SimpleMaterial

# Global constants and mappings (as before)
SBE_COLORS = {
    # Walls: a warm off-white with subtle cream tones.
    "walls": [0.8, 0.8, 0.8],
    # Interior walls: almost as light as exterior, with a slightly softer tone.
    "interior_walls": [0.99, 0.98, 0.97],
    # Roofs: using a darker tone here for contrast.
    "roofs": [0.2, 0.2, 0.2],
    # Ceilings: crisp pure white.
    "ceilings": [1.0, 1.0, 1.0],
    # Exterior floors: a light, cool concrete tone.
    "exterior_floors": [0.90, 0.88, 0.85],
    # Interior floors: a warm, soft wood-inspired tone.
    "interior_floors": [0.95, 0.92, 0.90],
    # Air walls: nearly pure white.
    "air_walls": [0.99, 0.99, 0.99],
    # Apertures (windows): a pastel blueish-grey.
    "apertures": [0.7, 0.75, 0.92],
    # Interior apertures (skylights): a slightly lighter version.
    "interior_apertures": [0.85, 0.88, 0.92],
    # Doors: kept dark as accent details.
    "doors": [0.20, 0.18, 0.16],
    # Interior doors: a soft dark gray.
    "interior_doors": [0.30, 0.28, 0.26],
    # Outdoor shades: a light neutral gray.
    "outdoor_shades": [0.65, 0.65, 0.65],
    # Indoor shades: a very light, airy gray.
    "indoor_shades": [0.75, 0.75, 0.75],
    # Shade meshes: a balanced mid-gray.
    "shade_meshes": [0.55, 0.55, 0.55],
}
FACE_TYPE_MAPPING = {
    "Wall": "walls",
    "RoofCeiling": "roofs",
    "Floor": "interior_floors",  # Default; updated later based on boundary conditions
}
APERTURE_OFFSET = 0.1

def get_rotation_matrix(rotate=False):
    """Return a rotation matrix. If rotate==True, rotate -90° about X-axis; otherwise, identity."""
    if rotate:
        return trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])
    else:
        return np.eye(4)

def compute_faces(num_vertices):
    """Return triangle fan faces for a polygon with num_vertices."""
    return np.array([[0, i, i + 1] for i in range(1, num_vertices - 1)], dtype=np.int64)

# --- Vectorized processing functions ---
def process_aperture_vectorized(aperture, face_type, rotation_matrix):
    """
    Process an aperture in a vectorized way:
      - Rotate its vertices,
      - Compute its normal (from the first triangle),
      - Offset all vertices along that normal,
      - Compute local faces.
    Returns (vertices, faces, color, category) or (None, None, None, None) if invalid.
    """
    aperture_geometry = aperture.get("geometry", {})
    boundary = aperture_geometry.get("boundary", [])
    if len(boundary) < 3:
        return None, None, None, None

    vertices = np.array(boundary, dtype=np.float32)
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))
    tri = rotated[0:3].reshape(1, 3, 3)
    normal = trimesh.triangles.normals(tri)[0]
    normal = trimesh.unitize(normal)
    rotated += APERTURE_OFFSET * normal

    if face_type == "Wall":
        category = "apertures"
    elif face_type == "RoofCeiling":
        category = "interior_apertures"
    else:
        category = "apertures"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    return rotated, local_faces, color, category

def process_face_vectorized(face, rotation_matrix):
    """
    Process a face by:
      - Rotating its vertices,
      - Computing local faces,
      - Determining its category,
      - And processing any attached apertures.
    Returns a dictionary mapping category names to lists of (vertices, faces, color) tuples.
    """
    results = {cat: [] for cat in SBE_COLORS.keys()}
    geometry = face.get("geometry", {})
    boundary = geometry.get("boundary", [])
    if len(boundary) < 3:
        return results

    vertices = np.array(boundary, dtype=np.float32)
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))
    face_type = face.get("face_type", "Wall")
    category = FACE_TYPE_MAPPING.get(face_type, "walls")
    bc = face.get("boundary_condition", {}).get("type", "")
    if face_type == "Floor":
        category = "exterior_floors" if bc == "Ground" else "interior_floors"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    results[category].append((rotated, local_faces, color))

    for aperture in face.get("apertures", []):
        ap_vertices, ap_faces, ap_color, ap_cat = process_aperture_vectorized(aperture, face_type, rotation_matrix)
        if ap_vertices is not None:
            results[ap_cat].append((ap_vertices, ap_faces, ap_color))
    return results

def process_room_vectorized(room, rotation_matrix):
    """
    Process all faces in a room.
    Returns a dictionary mapping category names to lists of (vertices, faces, color) tuples.
    """
    room_results = {cat: [] for cat in SBE_COLORS.keys()}
    for face in room.get("faces", []):
        face_dict = process_face_vectorized(face, rotation_matrix)
        for cat, items in face_dict.items():
            room_results[cat].extend(items)
    return room_results

def combine_category_geometry(items):
    """
    Given a list of items (each a tuple of (vertices, faces, color)) for one category,
    combine them into one mesh by concatenating vertices and faces.
    Returns a trimesh.Trimesh object.
    """
    if not items:
        return None
    all_vertices = []
    all_faces = []
    offset = 0
    for vertices, faces, _ in items:
        all_vertices.append(vertices)
        all_faces.append(faces + offset)
        offset += len(vertices)
    vertices_combined = np.vstack(all_vertices)
    faces_combined = np.vstack(all_faces)
    mesh = trimesh.Trimesh(vertices=vertices_combined, faces=faces_combined, process=False)
    # Set vertex colors (for debugging or fallback). This may later be overridden by materials.
    mesh.visual.vertex_colors = items[0][2]
    return mesh

# --- Build Scene Graph with Separate Mesh Nodes and Assigned Materials ---
def build_scene_from_category_meshes(combined_results):
    """
    For each category, combine its items into one mesh, assign a simple material,
    set the mesh's name to the category, and add it to a scene as a separate node.
    Returns a trimesh.Scene.
    """
    scene = trimesh.Scene()
    for cat, items in combined_results.items():
        mesh = combine_category_geometry(items)
        if mesh is not None:
            # Set the mesh name to the category.
            mesh.name = cat
            # Convert the normalized SBE_COLORS to 0-255 for a simple material.
            color = (np.array(SBE_COLORS[cat]) * 255).astype(np.uint8).tolist()
            # Assign a simple material to the mesh.
            mesh.visual.material = SimpleMaterial(color=color)
            # Add the mesh to the scene with the node name equal to the category.
            scene.add_geometry(mesh, node_name=cat, geom_name=cat)
    return scene

# --- Main Function ---
def main():
    # Configuration & Data Loading
    path_to_json = '../public/jättesten_model.hbjson'
    output_path = '../public/cachedGLB/demo.glb'
    rotation_matrix = get_rotation_matrix(rotate=False)  # Change to True if needed

    with open(path_to_json, 'r') as f:
        hbjson = json.load(f)
    rooms = hbjson.get("rooms", [])

    # Accumulate geometry per category.
    combined_results = {cat: [] for cat in SBE_COLORS.keys()}
    t0 = time.perf_counter()
    for room in rooms:
        room_geom = process_room_vectorized(room, rotation_matrix)
        for cat, items in room_geom.items():
            combined_results[cat].extend(items)
    t1 = time.perf_counter()
    print(f"Accumulated geometry from all rooms in {t1 - t0:.4f} seconds")

    # Build a scene with separate mesh nodes.
    scene = build_scene_from_category_meshes(combined_results)
    t2 = time.perf_counter()
    scene.export(output_path)
    t3 = time.perf_counter()
    print(f"Exported GLB file with scene graph: {output_path}")
    print(f"Scene assembly took {t2 - t1:.4f} seconds, export took {t3 - t2:.4f} seconds")

if __name__ == "__main__":
    main()


Accumulated geometry from all rooms in 0.3551 seconds
Exported GLB file with scene graph: ../public/cachedGLB/demo.glb
Scene assembly took 0.0180 seconds, export took 0.0061 seconds


In [6]:
import pygltflib

# Load the GLB file
glb_path = "../public/urban_district.glb"
glb = pygltflib.GLTF2().load(glb_path)

num_meshes = len(glb.meshes)
print(f"Number of meshes: {num_meshes}")

# Function to check if a mesh has a PBR material
def mesh_has_pbr_material(mesh):
    for primitive in mesh.primitives:
        material = glb.materials[primitive.material]
        if material.pbrMetallicRoughness:
            return True
    return False

# Check each mesh for PBR material
for i, mesh in enumerate(glb.meshes):
    has_pbr_material = mesh_has_pbr_material(mesh)
    print(f"Mesh {i} has PBR material: {has_pbr_material}")

# Function to check and fix material indices
def check_and_fix_material_indices(meshes, materials):
    fixed = False
    for mesh in meshes:
        for primitive in mesh.primitives:
            if primitive.material >= len(materials):
                print(f"Invalid material index {primitive.material} found. Fixing it to 0.")
                primitive.material = 0  # Assign a default valid material index
                fixed = True
    return fixed

# Check and fix material indices
fixed = check_and_fix_material_indices(glb.meshes, glb.materials)
if fixed:
    print("Some material indices were invalid and have been fixed.")
else:
    print("All material indices are valid.")
# Function to print mesh names, corresponding material indices, and material properties
def print_mesh_material_info(meshes, materials):
    for i, mesh in enumerate(meshes):
        print(f"Mesh {i} - Name: {mesh.name}")
        for j, primitive in enumerate(mesh.primitives):
            material_index = primitive.material
            if material_index < len(materials):
                material = materials[material_index]
                print(f"  Primitive {j} - Material Index: {material_index}")
                print(f"    Name: {material.name}")
                print(f"    Base Color Factor: {material.pbrMetallicRoughness.baseColorFactor}")
                print(f"    Metallic Factor: {material.pbrMetallicRoughness.metallicFactor}")
                print(f"    Roughness Factor: {material.pbrMetallicRoughness.roughnessFactor}")
            else:
                print(f"  Primitive {j} - Invalid Material Index: {material_index}")

# Print mesh material information
print_mesh_material_info(glb.meshes, glb.materials)

Number of meshes: 5
Mesh 0 has PBR material: True
Mesh 1 has PBR material: True
Mesh 2 has PBR material: True
Mesh 3 has PBR material: True
Mesh 4 has PBR material: True
All material indices are valid.
Mesh 0 - Name: walls
  Primitive 0 - Material Index: 0
    Name: None
    Base Color Factor: [0.4, 0.4, 0.4, 1.0]
    Metallic Factor: 1.0
    Roughness Factor: 0.9036020036098448
Mesh 1 - Name: roofs
  Primitive 0 - Material Index: 0
    Name: None
    Base Color Factor: [0.4, 0.4, 0.4, 1.0]
    Metallic Factor: 1.0
    Roughness Factor: 0.9036020036098448
Mesh 2 - Name: exterior_floors
  Primitive 0 - Material Index: 0
    Name: None
    Base Color Factor: [0.4, 0.4, 0.4, 1.0]
    Metallic Factor: 1.0
    Roughness Factor: 0.9036020036098448
Mesh 3 - Name: interior_floors
  Primitive 0 - Material Index: 0
    Name: None
    Base Color Factor: [0.4, 0.4, 0.4, 1.0]
    Metallic Factor: 1.0
    Roughness Factor: 0.9036020036098448
Mesh 4 - Name: apertures
  Primitive 0 - Material Index: 0

In [7]:
import pygltflib
from pygltflib import GLTF2, Material, PbrMetallicRoughness

# Path to your input GLB file.
glb_path = "../public/urban_district.glb"
output_path = "../public/urban_district_modified.glb"

# Load the GLB file.
gltf = GLTF2().load(glb_path)

print(f"Number of meshes: {len(gltf.meshes)}")

# Define a mapping from mesh name (lowercase) to desired PBR material properties.
# Each property is defined as a dictionary with keys: baseColorFactor, metallicFactor, roughnessFactor.
# The baseColorFactor should be an array of four numbers (RGBA) with values between 0 and 1.
material_specs = {
    "walls": {
        "baseColorFactor": [240/255, 230/255, 220/255, 1.0],  # warm off-white
        "metallicFactor": 0.0,
        "roughnessFactor": 0.8,
    },
    "roofs": {
        "baseColorFactor": [250/255, 245/255, 235/255, 1.0],  # even lighter for roofs
        "metallicFactor": 0.0,
        "roughnessFactor": 0.5,
    },
    "exterior_floors": {
        "baseColorFactor": [230/255, 230/255, 220/255, 1.0],  # light concrete tone
        "metallicFactor": 0.0,
        "roughnessFactor": 0.7,
    },
    "interior_floors": {
        "baseColorFactor": [240/255, 235/255, 220/255, 1.0],  # light wood tone
        "metallicFactor": 0.0,
        "roughnessFactor": 0.7,
    },
    "apertures": {
        "baseColorFactor": [200/255, 210/255, 230/255, 1.0],  # pastel blueish-grey for windows
        "metallicFactor": 0.0,
        "roughnessFactor": 0.3,
    }
}

# Create new materials based on material_specs.
new_materials_list = []
material_mapping = {}  # mapping from category name to its new material index
for key, spec in material_specs.items():
    mat = Material(
        name=key,
        pbrMetallicRoughness=PbrMetallicRoughness(
            baseColorFactor=spec["baseColorFactor"],
            metallicFactor=spec["metallicFactor"],
            roughnessFactor=spec["roughnessFactor"]
        )
    )
    new_materials_list.append(mat)
    # Record the index of this new material.
    material_mapping[key] = len(new_materials_list) - 1

# Optionally, you can also create a default material for meshes with an unrecognized name.
default_material = Material(
    name="default",
    pbrMetallicRoughness=PbrMetallicRoughness(
        baseColorFactor=[1.0, 1.0, 1.0, 1.0],
        metallicFactor=0.0,
        roughnessFactor=1.0
    )
)
new_materials_list.append(default_material)
material_mapping["default"] = len(new_materials_list) - 1

# Replace the materials in the GLTF with our new materials.
gltf.materials = new_materials_list

# For each mesh, update each primitive's material index based on the mesh name.
for mesh in gltf.meshes:
    # Use the mesh name to look up the desired material.
    if mesh.name:
        key = mesh.name.lower()
        mat_index = material_mapping.get(key, material_mapping["default"])
    else:
        mat_index = material_mapping["default"]

    for primitive in mesh.primitives:
        primitive.material = mat_index

# Optionally, you can print out the new material assignments:
def print_mesh_material_info(meshes, materials):
    for i, mesh in enumerate(meshes):
        print(f"Mesh {i} - Name: {mesh.name}")
        for j, primitive in enumerate(mesh.primitives):
            material_index = primitive.material
            if material_index < len(materials):
                material = materials[material_index]
                print(f"  Primitive {j} - Material Index: {material_index}")
                print(f"    Material Name: {material.name}")
                print(f"    Base Color Factor: {material.pbrMetallicRoughness.baseColorFactor}")
                print(f"    Metallic Factor: {material.pbrMetallicRoughness.metallicFactor}")
                print(f"    Roughness Factor: {material.pbrMetallicRoughness.roughnessFactor}")
            else:
                print(f"  Primitive {j} - Invalid Material Index: {material_index}")

print_mesh_material_info(gltf.meshes, gltf.materials)

# Save the modified GLB.
gltf.save_binary(output_path)
print(f"Modified GLB saved to: {output_path}")


Number of meshes: 5
Mesh 0 - Name: walls
  Primitive 0 - Material Index: 0
    Material Name: walls
    Base Color Factor: [0.9411764705882353, 0.9019607843137255, 0.8627450980392157, 1.0]
    Metallic Factor: 0.0
    Roughness Factor: 0.8
Mesh 1 - Name: roofs
  Primitive 0 - Material Index: 1
    Material Name: roofs
    Base Color Factor: [0.9803921568627451, 0.9607843137254902, 0.9215686274509803, 1.0]
    Metallic Factor: 0.0
    Roughness Factor: 0.5
Mesh 2 - Name: exterior_floors
  Primitive 0 - Material Index: 2
    Material Name: exterior_floors
    Base Color Factor: [0.9019607843137255, 0.9019607843137255, 0.8627450980392157, 1.0]
    Metallic Factor: 0.0
    Roughness Factor: 0.7
Mesh 3 - Name: interior_floors
  Primitive 0 - Material Index: 3
    Material Name: interior_floors
    Base Color Factor: [0.9411764705882353, 0.9215686274509803, 0.8627450980392157, 1.0]
    Metallic Factor: 0.0
    Roughness Factor: 0.7
Mesh 4 - Name: apertures
  Primitive 0 - Material Index: 4
 

In [3]:
import json
import time
import numpy as np
import trimesh
from trimesh.visual.material import SimpleMaterial  # used for scene export (temporary)
from pygltflib import GLTF2, Material as GLTFMaterial, PbrMetallicRoughness

# --- Global constants and mappings ---
SBE_COLORS = {
    "walls": [0.8, 0.8, 0.8],
    "interior_walls": [0.99, 0.98, 0.97],
    "roofs": [0.2, 0.2, 0.2],
    "ceilings": [1.0, 1.0, 1.0],
    "exterior_floors": [0.90, 0.88, 0.85],
    "interior_floors": [0.95, 0.92, 0.90],
    "air_walls": [0.99, 0.99, 0.99],
    "apertures": [0.7, 0.75, 0.92],
    "interior_apertures": [0.85, 0.88, 0.92],
    "doors": [0.20, 0.18, 0.16],
    "interior_doors": [0.30, 0.28, 0.26],
    "outdoor_shades": [0.65, 0.65, 0.65],
    "indoor_shades": [0.75, 0.75, 0.75],
    "shade_meshes": [0.55, 0.55, 0.55],
}
FACE_TYPE_MAPPING = {
    "Wall": "walls",
    "RoofCeiling": "roofs",
    "Floor": "interior_floors",  # Updated later based on boundary conditions.
}
APERTURE_OFFSET = 0.1

# --- Utility Functions ---
def get_rotation_matrix(rotate=False):
    """Return a rotation matrix. If rotate is True, rotate -90° about the X-axis; otherwise, identity."""
    if rotate:
        return trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])
    else:
        return np.eye(4)

def compute_faces(num_vertices):
    """Return triangle fan faces for a polygon with num_vertices."""
    return np.array([[0, i, i + 1] for i in range(1, num_vertices - 1)], dtype=np.int64)

# --- Vectorized Processing Functions ---
def process_aperture_vectorized(aperture, face_type, rotation_matrix):
    """
    Process an aperture:
      - Rotate its vertices.
      - Compute its normal (from the first triangle).
      - Offset all vertices along that normal.
      - Compute local faces.
    Returns (vertices, faces, color, category) or (None, None, None, None) if invalid.
    """
    aperture_geometry = aperture.get("geometry", {})
    boundary = aperture_geometry.get("boundary", [])
    if len(boundary) < 3:
        return None, None, None, None

    vertices = np.array(boundary, dtype=np.float32)
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))
    tri = rotated[0:3].reshape(1, 3, 3)
    normal = trimesh.triangles.normals(tri)[0]
    normal = trimesh.unitize(normal)
    rotated += APERTURE_OFFSET * normal

    if face_type == "Wall":
        category = "apertures"
    elif face_type == "RoofCeiling":
        category = "interior_apertures"
    else:
        category = "apertures"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    return rotated, local_faces, color, category

def process_face_vectorized(face, rotation_matrix):
    """
    Process a face:
      - Rotate its vertices.
      - Compute local faces.
      - Determine its category.
      - Process any attached apertures.
    Returns a dictionary mapping category names to lists of (vertices, faces, color) tuples.
    """
    results = {cat: [] for cat in SBE_COLORS.keys()}
    geometry = face.get("geometry", {})
    boundary = geometry.get("boundary", [])
    if len(boundary) < 3:
        return results

    vertices = np.array(boundary, dtype=np.float32)
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))
    face_type = face.get("face_type", "Wall")
    category = FACE_TYPE_MAPPING.get(face_type, "walls")
    bc = face.get("boundary_condition", {}).get("type", "")
    if face_type == "Floor":
        category = "exterior_floors" if bc == "Ground" else "interior_floors"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    results[category].append((rotated, local_faces, color))

    for aperture in face.get("apertures", []):
        ap_vertices, ap_faces, ap_color, ap_cat = process_aperture_vectorized(aperture, face_type, rotation_matrix)
        if ap_vertices is not None:
            results[ap_cat].append((ap_vertices, ap_faces, ap_color))
    return results

def process_room_vectorized(room, rotation_matrix):
    """
    Process all faces in a room.
    Returns a dictionary mapping category names to lists of (vertices, faces, color) tuples.
    """
    room_results = {cat: [] for cat in SBE_COLORS.keys()}
    for face in room.get("faces", []):
        face_dict = process_face_vectorized(face, rotation_matrix)
        for cat, items in face_dict.items():
            room_results[cat].extend(items)
    return room_results

def combine_category_geometry(items):
    """
    Given a list of items (each a tuple of (vertices, faces, color)) for one category,
    combine them into one mesh by concatenating vertices and faces.
    Returns a trimesh.Trimesh object.
    """
    if not items:
        return None
    all_vertices = []
    all_faces = []
    offset = 0
    for vertices, faces, _ in items:
        all_vertices.append(vertices)
        all_faces.append(faces + offset)
        offset += len(vertices)
    vertices_combined = np.vstack(all_vertices)
    faces_combined = np.vstack(all_faces)
    mesh = trimesh.Trimesh(vertices=vertices_combined, faces=faces_combined, process=False)
    # For now, set the vertex color (this is temporary and will be replaced by materials).
    mesh.visual.vertex_colors = items[0][2]
    return mesh

def build_scene_from_category_meshes(combined_results):
    """
    For each category, combine its items into one mesh, assign a simple material using SimpleMaterial,
    set the mesh's name to the category, and add it to a scene as a separate node.
    Returns a trimesh.Scene.
    """
    scene = trimesh.Scene()
    for cat, items in combined_results.items():
        mesh = combine_category_geometry(items)
        if mesh is not None:
            # Set the mesh name to the category.
            mesh.name = cat
            # Convert the normalized SBE_COLORS to 0-255.
            color = (np.array(SBE_COLORS[cat]) * 255).astype(np.uint8).tolist()
            # Assign a simple material.
            mesh.visual.material = SimpleMaterial(color=color)
            scene.add_geometry(mesh, node_name=cat)
    return scene

# --- GLB Post-Processing with pygltflib ---
def postprocess_glb(glb_path, output_path):
    """
    Load a GLB file, create new PBR materials based on mesh names, and update the material indices.
    Save the modified GLB.
    """
    gltf = GLTF2().load(glb_path)
    
    # Define material specs: mapping from category name to PBR properties (normalized values)
    material_specs = {
        "walls": {
            "baseColorFactor": [0.8, 0.8, 0.8, 1.0],
            "metallicFactor": 0.0,
            "roughnessFactor": 1,
            "doubleSided": True,
        },
        "roofs": {
            "baseColorFactor": [250/255, 245/255, 235/255, 1.0],
            "metallicFactor": 0.0,
            "roughnessFactor": 1,
            "doubleSided": True,
        },
        "exterior_floors": {
            "baseColorFactor": [230/255, 230/255, 220/255, 1.0],
            "metallicFactor": 0.0,
            "roughnessFactor": 1,
            "doubleSided": True,
        },
        "interior_floors": {
            "baseColorFactor": [240/255, 235/255, 220/255, 1.0],
            "metallicFactor": 0.0,
            "roughnessFactor": 1,
            "doubleSided": True,
        },
        "apertures": {
            "baseColorFactor": [0.2, 0.2, 0.8, 1.0],
            "metallicFactor": 0.0,
            "roughnessFactor": 1,
            "doubleSided": True,
        }
    }
    # Create new GLTF materials based on material_specs.
    new_materials = []
    material_mapping = {}
    for cat, spec in material_specs.items():
        mat = GLTFMaterial(
            name=cat,
            pbrMetallicRoughness=PbrMetallicRoughness(
                baseColorFactor=spec["baseColorFactor"],
                metallicFactor=spec["metallicFactor"],
                roughnessFactor=spec["roughnessFactor"]
                
            )
        )
        new_materials.append(mat)
        material_mapping[cat] = len(new_materials) - 1

    # Create a default material.
    default_mat = GLTFMaterial(
        name="default",
        pbrMetallicRoughness=PbrMetallicRoughness(
            baseColorFactor=[1.0, 1.0, 1.0, 1.0],
            metallicFactor=0.0,
            roughnessFactor=1.0
        )
    )
    new_materials.append(default_mat)
    material_mapping["default"] = len(new_materials) - 1

    # Replace the materials in the glTF.
    gltf.materials = new_materials

    # For each mesh, set the primitive material index based on the mesh name.
    for mesh in gltf.meshes:
        key = mesh.name.lower() if mesh.name else "default"
        mat_index = material_mapping.get(key, material_mapping["default"])
        for primitive in mesh.primitives:
            primitive.material = mat_index

    gltf.save_binary(output_path)
    print(f"Post-processed GLB saved to: {output_path}")

# --- Combined Workflow Main Function ---
def main():
    # Configuration & Data Loading
    hbjson_path = "../public/urban_district.hbjson"
    intermediate_glb = hbjson_path.replace(".hbjson", "_intermediate.glb")
    final_glb = '../public/cachedGLB/demo.glb'
    #final_glb = hbjson_path.replace(".hbjson", ".glb")
    rotation_matrix = get_rotation_matrix(rotate=False)  # Set to True if needed

    with open(hbjson_path, 'r') as f:
        hbjson = json.load(f)
    rooms = hbjson.get("rooms", [])

    # Accumulate geometry per category.
    combined_results = {cat: [] for cat in SBE_COLORS.keys()}
    t0 = time.perf_counter()
    for room in rooms:
        room_geom = process_room_vectorized(room, rotation_matrix)
        for cat, items in room_geom.items():
            combined_results[cat].extend(items)
    t1 = time.perf_counter()
    print(f"Accumulated geometry from all rooms in {t1 - t0:.4f} seconds")

    # Build a scene with one mesh per category.
    scene = build_scene_from_category_meshes(combined_results)
    t2 = time.perf_counter()
    scene.export(intermediate_glb)
    t3 = time.perf_counter()
    print(f"Exported intermediate GLB: {intermediate_glb}")
    print(f"Scene assembly took {t2 - t1:.4f} seconds, export took {t3 - t2:.4f} seconds")

    # Post-process the GLB with pygltflib to assign proper PBR materials.
    postprocess_glb(intermediate_glb, final_glb)

if __name__ == "__main__":
    main()


Accumulated geometry from all rooms in 0.9141 seconds
Exported intermediate GLB: ../public/urban_district_intermediate.glb
Scene assembly took 0.0572 seconds, export took 0.0147 seconds
Post-processed GLB saved to: ../public/cachedGLB/demo.glb
